In [1]:
import numpy as np
import os
import cv2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle

#importing all the libraries needed for the project

In [2]:
path = 'D:/E/'
x_train=[]
y_train=[]
x_test=[]
os.chdir(path+'train/')

#declaring the path and variables

In [3]:
for i in os.listdir():
    for j in os.listdir(i):
        temp=cv2.imread(path+'train/'+i+'/'+j)
        temp=cv2.resize(temp,(128,128))
        x_train.append(temp)
        y_train.append(i)
        
#reading the images to x_train and catagory to y_train

In [4]:
os.chdir(path+'test/')

In [5]:
for i in os.listdir():
    temp=cv2.imread(path+'test/'+i)
    temp=cv2.resize(temp,(128,128))
    x_test.append(temp)
    
#reading the images to x_test

In [6]:
np.unique(y_train).shape # Numbers of unique classes in y

(12,)

In [7]:
pickle_out = open("D:/E/Sachin/x_train.pickle","wb")
pickle.dump(x_train,pickle_out)
pickle_out.close()
pickle_out = open("D:/E/Sachin/x_test.pickle","wb")
pickle.dump(x_test,pickle_out)
pickle_out.close()
pickle_out = open("D:/E/Sachin/y_train.pickle","wb")
pickle.dump(y_train,pickle_out)
pickle_out.close()

#saving the file as pickle file for later use

In [8]:
pickle_in = open("D:/E/Sachin/x_train.pickle","rb")
x_train = pickle.load(pickle_in) 
pickle_in.close()
pickle_in = open("D:/E/Sachin/y_train.pickle","rb")
y_train = pickle.load(pickle_in) 
pickle_in.close()

#using the pickle file saved before

In [9]:
lb_make = LabelEncoder()
y_train = lb_make.fit_transform(y_train)

In [10]:
np.unique(y_train).shape

(12,)

In [11]:
y_train1 = tf.keras.utils.to_categorical(y_train, num_classes=12) 
# as from previous step we know unique classes = 12, so choosing number of classes as 12

In [12]:
x_train=np.asarray(x_train)
x_train=x_train/255.0

In [13]:
X1_train, X1_test, y1_train, y1_test = train_test_split(x_train, y_train1, test_size=0.3, random_state=42)

#spliting the train data again to train and test, test data here represents the validation data

In [14]:
model = tf.keras.Sequential()

#CNN model

In [15]:
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu',input_shape=(128,128,3)))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides =(2,2), padding='valid'))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization(axis=3))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides =(2,2), padding='valid'))
model.add(tf.keras.layers.Dropout(0.2))


model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides =(2,2), padding='valid'))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization(axis=3))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides =(2,2), padding='valid'))
model.add(tf.keras.layers.Dropout(0.2))


model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization(axis=3))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides =(2,2), padding='valid'))
model.add(tf.keras.layers.Dropout(0.2))


model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.LayerNormalization())
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(12, activation='softmax'))

In [16]:
model.compile(loss='categorical_crossentropy',optimizer='SGD',lr=0.01,metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0

In [17]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='auto')
callback_list = [early_stopping]
model.fit(X1_train,y1_train,batch_size=32,callbacks=callback_list,epochs=20,validation_data=(X1_test,y1_test))

Train on 3325 samples, validate on 1425 samples
Epoch 1/20
3325/3325 [==============================] - 128s 38ms/sample - loss: 2.4911 - accuracy: 0.1398 - val_loss: 2.9969 - val_accuracy: 0.0730
Epoch 2/20
3325/3325 [==============================] - 118s 36ms/sample - loss: 2.1691 - accuracy: 0.2460 - val_loss: 2.9826 - val_accuracy: 0.1354
Epoch 3/20
3325/3325 [==============================] - 116s 35ms/sample - loss: 1.9944 - accuracy: 0.3233 - val_loss: 3.2361 - val_accuracy: 0.1277
Epoch 4/20
3325/3325 [==============================] - 108s 33ms/sample - loss: 1.7299 - accuracy: 0.4250 - val_loss: 4.0777 - val_accuracy: 0.0982
Epoch 5/20
3325/3325 [==============================] - 101s 30ms/sample - loss: 1.4748 - accuracy: 0.5026 - val_loss: 2.5342 - val_accuracy: 0.2821
Epoch 6/20
3325/3325 [==============================] - 100s 30ms/sample - loss: 1.3111 - accuracy: 0.5612 - val_loss: 1.4569 - val_accuracy: 0.4793
Epoch 7/20
3325/3325 [==============================] - 10

In [18]:
scores = model.evaluate(X1_train, y1_train, verbose=0)
print("Train Accuracy: %.2f%%" % (scores[1]*100))

Train Accuracy: 86.41%


In [19]:
scores = model.evaluate(X1_test, y1_test, verbose=0)
print("Test Accuracy: %.2f%%" % (scores[1]*100))

Test Accuracy: 80.49%


In [20]:
from sklearn.metrics import confusion_matrix

# PREDICTIONS
y_pred = model.predict(X1_test)
y_class = np.argmax(y_pred, axis = 1) 
y_check = np.argmax(y1_test, axis = 1) 

cmatrix = confusion_matrix(y_check, y_class)

print(cmatrix)

#Confusion matrix for the model

[[ 18   0   0   0  14   0  50   0   2   0   0   3]
 [  0 108   8   0   0   1   0   1   1   0   1   0]
 [  0   3  78   0   3   0   0   0   0   0   0   2]
 [  0   0   0 186   0   1   0   1   2   0   2   1]
 [  2   0   1   0  57   2   1   0   0   0   0   2]
 [  0   4   9   2   8  96   1   2   0   0   0  21]
 [ 10   0   0   0   6   9 166   0   5   0   2   0]
 [  0   2   1   1   1   2   1  59   0   0   0   3]
 [  0   4   0   0   2   1   1   1 121   0   0   6]
 [  0   2   1  10   0   3   0   1  14  21   4   2]
 [  0   5   5   0   0   0   1   4   0   2 133   1]
 [  0   2   2   0   1   1   0   8   0   0   0 104]]


In [21]:
from sklearn.metrics import classification_report 
report = classification_report(np.argmax(y_pred,axis=1),np.argmax(y1_test,axis=1)) 
print('Final Validation Results') 
print(report)

Final Validation Results
              precision    recall  f1-score   support

           0       0.21      0.60      0.31        30
           1       0.90      0.83      0.86       130
           2       0.91      0.74      0.82       105
           3       0.96      0.93      0.95       199
           4       0.88      0.62      0.73        92
           5       0.67      0.83      0.74       116
           6       0.84      0.75      0.79       221
           7       0.84      0.77      0.80        77
           8       0.89      0.83      0.86       145
           9       0.36      0.91      0.52        23
          10       0.88      0.94      0.91       142
          11       0.88      0.72      0.79       145

    accuracy                           0.80      1425
   macro avg       0.77      0.79      0.76      1425
weighted avg       0.85      0.80      0.82      1425



In [30]:
datagen = ImageDataGenerator(rotation_range = 180,zoom_range = 0.1,
                             width_shift_range = 0.1,height_shift_range = 0.1,
                             horizontal_flip = True,vertical_flip = True,
                             channel_shift_range=(50),
                             fill_mode='nearest')
datagen.fit(X1_train)

In [31]:
hist = model.fit_generator(datagen.flow(X1_train, y1_train, batch_size=32),
                   steps_per_epoch=X1_train.shape[0] // 32,
                   epochs=10,
                   verbose=2,
                   validation_data=(X1_test, y1_test))

Epoch 1/10
103/103 - 156s - loss: 2.2667 - accuracy: 0.1937 - val_loss: 2.4919 - val_accuracy: 0.0961
Epoch 2/10
103/103 - 156s - loss: 2.2830 - accuracy: 0.1770 - val_loss: 2.4809 - val_accuracy: 0.1004
Epoch 3/10
103/103 - 149s - loss: 2.2790 - accuracy: 0.1828 - val_loss: 2.5056 - val_accuracy: 0.1116
Epoch 4/10
103/103 - 149s - loss: 2.2900 - accuracy: 0.1877 - val_loss: 2.4823 - val_accuracy: 0.1614
Epoch 5/10
103/103 - 148s - loss: 2.2753 - accuracy: 0.1792 - val_loss: 2.4700 - val_accuracy: 0.1151
Epoch 6/10
103/103 - 148s - loss: 2.2613 - accuracy: 0.1953 - val_loss: 2.5106 - val_accuracy: 0.0961
Epoch 7/10
103/103 - 135s - loss: 2.2739 - accuracy: 0.1877 - val_loss: 2.4770 - val_accuracy: 0.1256
Epoch 8/10
103/103 - 134s - loss: 2.2519 - accuracy: 0.1998 - val_loss: 2.5118 - val_accuracy: 0.1074
Epoch 9/10
103/103 - 133s - loss: 2.2767 - accuracy: 0.1977 - val_loss: 2.4557 - val_accuracy: 0.1221
Epoch 10/10
103/103 - 140s - loss: 2.2744 - accuracy: 0.1852 - val_loss: 2.4443 - 

In [32]:
scores = model.evaluate(X1_train, y1_train, verbose=0)
print("Train Accuracy: %.2f%%" % (scores[1]*100))

Train Accuracy: 14.02%


In [33]:
scores = model.evaluate(X1_test, y1_test, verbose=0)
print("Test Accuracy: %.2f%%" % (scores[1]*100))

Test Accuracy: 12.42%


In [34]:
#PREDICTIONS
y_pred = model.predict(X1_test)
y_class = np.argmax(y_pred, axis = 1) 
y_check = np.argmax(y1_test, axis = 1) 

cmatrix = confusion_matrix(y_check, y_class)

print(cmatrix)

#model stuck with local minima and didnt move from the local minima, hence the expected output isnt acheived.
#ideally model should have performed better with the data image generation, due to computational constraint the model isnt re-run. 
#(having issues with running it in google colab, due to large data file)

[[  2   0   0  11   0   0  26   0  48   0   0   0]
 [  0   0   0  10   0   0   5   0 105   0   0   0]
 [  0   0   0   2   0   0   6   0  78   0   0   0]
 [  0   0   0   9   0   0  15   0 169   0   0   0]
 [  0   0   0   6   0   0  13   0  46   0   0   0]
 [  0   0   0   5   0   0  13   0 125   0   0   0]
 [  2   0   0   6   0   0  48   0 142   0   0   0]
 [  0   0   0   5   0   0   6   0  59   0   0   0]
 [  1   0   0   3   0   0  15   0 117   0   0   0]
 [  0   0   0   0   0   0   4   0  54   0   0   0]
 [  0   0   0   8   0   0  20   0 122   0   1   0]
 [  0   0   0   2   0   0   6   0 110   0   0   0]]


In [35]:
report = classification_report(np.argmax(y_pred,axis=1),np.argmax(y1_test,axis=1)) 
print('Final Validation Results') 
print(report)

Final Validation Results
              precision    recall  f1-score   support

           0       0.02      0.40      0.04         5
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.05      0.13      0.07        67
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.24      0.27      0.26       177
           7       0.00      0.00      0.00         0
           8       0.86      0.10      0.18      1175
           9       0.00      0.00      0.00         0
          10       0.01      1.00      0.01         1
          11       0.00      0.00      0.00         0

    accuracy                           0.12      1425
   macro avg       0.10      0.16      0.05      1425
weighted avg       0.74      0.12      0.18      1425



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
